# Car Analytics
This project leverages the deep learning library __Tensorflow__ (with GPU) and analyzes cars in the real-time.
It includes the following modules:
* __High Speed Car Detection (HSCD):__ This module analyzes the incoming frames and detects the cars moving with a speed greater than the specified threshold value. 
* __Cars Moving in Wrong Direction (CMWD):__ Given an input frame, this module detects the cars which are moving in wrong direction.
* __Traffic Signal Violation (TSV):__ The purpose of this module is to detect and flag all the cars violating a particular given traffic signal.
* __Counting Cars:__ This module count the cars passing a region in a given frame. 

* __All the modules run simultaneously in real time and capture the above events individually.__

Note: User configurable parameters for this project are in config.ini file in the base directory(/home/ubuntu/notebook/video-analytics/config/config.ini) config folder.

## Steps to call API functions

### Step 1: Call class imports

In [1]:
from car_analytics import CarAnalytics

### Step 2: Setup video path

In [3]:
# Video path, could be a local video as well as an RTSP feed.
# path_to_video = 'rtsp://<ip-address>/demoVideo.mp4' 
path_to_video = 'data/red_light_sim_1.mp4' # Local video

### Step 3. Make instance variable for Car Analytics class with path to video as a parameter
#### function: CarAnalytics(path_to_video)
This function takes in a path to video file location to instantiate the CarAnalytics class.

It creates an object of the CarAnalytics class


In [4]:
# Video path, could be a local video as well as an RTSP feed.
video_object = CarAnalytics(path_to_video)


***Initialization Complete!***


Please proceed for individual module setup



### Step 3: Setup counting cars module
This  module does not requires any parameters to setup.

#### function: setup_counting_cars()
Sets up and counting cars module for given location or video.

In [5]:
video_object.setup_counting_cars()

### Step 4: Setup high speed detection module
This module requires user parameters from the user as a part of calibration procedure. 

#### function: setup_high_speed_car_detection()
User can either input parameters during API call or can pass the same via config.ini file in the config folder since these are project based parameter.

User also needs to __setup an roi__ which pops up immediately after running this function. All the HSCD parameters are saved as per video name or location name.

#### Required parameters:
* distance
* frame rate
* speed limit threshold

In [6]:
distance = 10
frameRate = 15
speedLimit = 10

In [7]:
video_object.setup_high_speed_detection(distance, frameRate, speedLimit)

AttributeError: CarAnalytics instance has no attribute 'setup_high_speed_detection'

### ROI setup instructions for high speed car detection module
An roi with instructions (as shown below) pops up when a user runs the __setup_high_speed_car_detection()__ function.

In [7]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "RoI_Instruction_Image_HSCD.jpg")

### Step 5: Setup cars moving in wrong direction module

#### function: setup_wrong_direction_detection()
User needs to setup an roi which pops up immediately after running this function. All the CMWD parameters are saved as per video name or location name.

In [8]:
video_object.setup_wrong_direction_detection()

### ROI setup instructions for cars moving in wrong direction module
An roi with instructions (as shown below) pops up when a user runs the __setup_wrong_direction_detection()__ function.

<img src="RoI_Instruction_Image_WD.jpg">

### Step 6: Setup traffic signal violation module

#### function: setup_traffic_signal_violation_detection()
User needs to setup an roi which pops up immediately after running this function. All the TSV parameters are saved as per video name or location name.

In [9]:
video_object.setup_traffic_signal_violation_detection()

### ROI setup instructions for  module traffic signal violation module
For this particular module, there are two rois that pop up when a user runs the __setup_traffic_signal_violation_detection()__ function.

#### ROI for traffic signal selection

<img src="RoI_Instruction_Image_RL.jpg">

#### ROI for direction detection

In [10]:
Image(url= "RoI_Instruction_Image_VZ.jpg")

____

### Step 7: Notify user for the output path
The project runs in the real time, therefore once the setup for modules has been done, this function can be run in parallel to get the path where the output will be written. 

*A central poller which provides real-time alerts to the __Atlantis UI__ should looks for the poller present in the below mentioned path and start giving out alerts.*

#### function: show_output_path()
Sends one time notification to the UI once all the calibration has been done.

In [11]:
video_object.show_output_path()

THE OUTPUT FOR THIS RUN OF THE API WILL BE AT: /home/ubuntu/notebook/video-analytics/output/bangalore/realtime_car_analytics/red_light_sim_1


### Step 8: Run modules (main process start-up)

#### function: run()
All the modules run independent of each other.
Once modules have been setup, we call the tensorflow program entry point __run()__.

In [12]:
video_object.run()

Parsing darkflow/cfg/yolo.cfg
Loading darkflow/bin/yolo.weights ...
Successfully identified 203934260 bytes
Finished in 0.0252249240875s
Model has a coco model name, loading coco labels.

Building net ...
Source | Train? | Layer description                | Output size
-------+--------+----------------------------------+---------------
       |        | input                            | (?, 608, 608, 3)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 608, 608, 32)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 304, 304, 32)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 304, 304, 64)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 152, 152, 64)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 152, 152, 128)
 Load  |  Yep!  | conv 1x1p0_1  +bnorm  leaky      | (?, 152, 152, 64)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 152, 152, 128)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 76, 76, 128)
 Load  |  Yep!  | conv 

### Expected Output

<img src="output.jpg">

____

____

## How to use the results aggregated by our API?

### 1. Sample Alert Poller
* All events (*HSCD, Cars in Wrong Direction, Traffic Signal Violation*) happen in real time. 

* As soon as an event happens, it is logged into a time-stamped alert poller file. 

* A separate / independent python process / function can run and keep checking this file for entries and can then send out notificaions to the __Atlantis UI__.

* A sample implementation has been done below.

### function: path_to_alert_poller()
The purpose of exposing this function is to directly provide the path where the events are being logged.

In [13]:
from pprint import pprint
import os
import json

fname = video_object.path_to_alert_poller()

poll_content = []
if not os.path.isfile(fname):
    print("alerts for this has not been generated yet, please try again!")

else:
    print("Alerts have been generated, sample poller ready!\n")
    with open(fname) as fp:
        for line in fp:
            poll_content.append(line.strip())
    pprint(poll_content)

Alerts have been generated, sample poller ready!

['/home/ubuntu/notebook/video-analytics/output/bangalore/realtime_car_analytics/red_light_sim_1/hscd/2017_12_03_17_12_09_videos/4/2017_12_03_17_12_09_4.json',
 '/home/ubuntu/notebook/video-analytics/output/bangalore/realtime_car_analytics/red_light_sim_1/cmwd/2017_12_03_17_12_09_videos/4/2017_12_03_17_12_09_4.json',
 '/home/ubuntu/notebook/video-analytics/output/bangalore/realtime_car_analytics/red_light_sim_1/hscd/2017_12_03_17_12_09_videos/9/2017_12_03_17_12_09_9.json',
 '/home/ubuntu/notebook/video-analytics/output/bangalore/realtime_car_analytics/red_light_sim_1/cmwd/2017_12_03_17_12_09_videos/9/2017_12_03_17_12_09_9.json',
 '/home/ubuntu/notebook/video-analytics/output/bangalore/realtime_car_analytics/red_light_sim_1/tsv/2017_12_03_17_12_09_videos/13/2017_12_03_17_12_09_13.json',
 '/home/ubuntu/notebook/video-analytics/output/bangalore/realtime_car_analytics/red_light_sim_1/tsv/2017_12_03_17_12_09_videos/17/2017_12_03_17_12_09_17.j

### 2. Event information (ALERT generation)
* Every event has a detailed information associated with it.

* Below we manually pickup events for *HSCD, Cars in Wrong DIrection and Traffic Signal Violation* to show how it would be __consumed__ by Atlantis platform.

In [14]:
def process_event(data):
    if data['module_name'] == "hscd":
        print "Tagged Car ID: {} has done a SPEED VIOLAION at location: {}, at time: {}. Speed Detected: {} Kmph. Speed Limit: {} Kmph. For more details, check url: https://<server-ip>{}".format(data['tagged_car_id'], data['location'], data['time'], data['speed_detected'], data['speed_threshold'], data['url'])
    elif data['module_name'] == "cmwd":
        print "Tagged Car ID: {} was found MOVING IN WRONG DIRECTION at location: {}, at time: {}. For more details, check url: https://<server-ip>{}".format(data['tagged_car_id'], data['location'], data['time'], data['url'])
    elif data['module_name'] == "tsv":
        print "Tagged Car ID: {} was found VIOLATING TRAFFIC SIGNAL at location: {}, at time: {}. For more details, check url: https://<server-ip>{}".format(data['tagged_car_id'], data['location'], data['time'], data['url'])

### 1. Sample for HSCD

* Load HSCD event 
* Car ID: 4

__*An event for HSCD is picked up from poller and then processed by process_event() function.*__

In [15]:
data = json.load(open(poll_content[0]))
print(json.dumps(data, indent=2))

{
  "tagged_car_id": 4, 
  "speed_detected": 34, 
  "url": "/home/ubuntu/notebook/video-analytics/output/bangalore/realtime_car_analytics/red_light_sim_1/hscd/2017_12_03_17_12_09_videos/4/2017_12_03_17_12_09_4.mp4", 
  "location": "red_light_sim_1", 
  "time": "2017-12-03 17:12:19", 
  "module_name": "hscd", 
  "speed_threshold": 10
}


### Process event

In [16]:
process_event(data)

Tagged Car ID: 4 has done a SPEED VIOLAION at location: red_light_sim_1, at time: 2017-12-03 17:12:19. Speed Detected: 34 Kmph. Speed Limit: 10 Kmph. For more details, check url: https://<server-ip>/home/ubuntu/notebook/video-analytics/output/bangalore/realtime_car_analytics/red_light_sim_1/hscd/2017_12_03_17_12_09_videos/4/2017_12_03_17_12_09_4.mp4


### 2. Sample for Cars in Wrong Direction (CMWD)

* Load CMWD event 
* Car ID: 9

__*An event for CMWD is picked up from poller and then processed by process_event() function.*__

In [17]:
data = json.load(open(poll_content[3]))
print(json.dumps(data, indent=2))

{
  "url": "/home/ubuntu/notebook/video-analytics/output/bangalore/realtime_car_analytics/red_light_sim_1/cmwd/2017_12_03_17_12_09_videos/9/2017_12_03_17_12_09_9.mp4", 
  "module_name": "cmwd", 
  "tagged_car_id": 9, 
  "location": "red_light_sim_1", 
  "time": "2017-12-03 17:12:22"
}


### Process event

In [18]:
process_event(data)

Tagged Car ID: 9 was found MOVING IN WRONG DIRECTION at location: red_light_sim_1, at time: 2017-12-03 17:12:22. For more details, check url: https://<server-ip>/home/ubuntu/notebook/video-analytics/output/bangalore/realtime_car_analytics/red_light_sim_1/cmwd/2017_12_03_17_12_09_videos/9/2017_12_03_17_12_09_9.mp4


### 3. Sample for Traffic Signal Violation (TSV)

* Load TSV event 
* Car ID: 17

__*An event for TSV is picked up from poller and then processed by process_event() function.*__

In [19]:
data = json.load(open(poll_content[5]))
print(json.dumps(data, indent=2))

{
  "url": "/home/ubuntu/notebook/video-analytics/output/bangalore/realtime_car_analytics/red_light_sim_1/tsv/2017_12_03_17_12_09_videos/17/2017_12_03_17_12_09_17.mp4", 
  "module_name": "tsv", 
  "tagged_car_id": 17, 
  "location": "red_light_sim_1", 
  "time": "2017-12-03 17:12:31"
}


### Process event

In [20]:
process_event(data)

Tagged Car ID: 17 was found VIOLATING TRAFFIC SIGNAL at location: red_light_sim_1, at time: 2017-12-03 17:12:31. For more details, check url: https://<server-ip>/home/ubuntu/notebook/video-analytics/output/bangalore/realtime_car_analytics/red_light_sim_1/tsv/2017_12_03_17_12_09_videos/17/2017_12_03_17_12_09_17.mp4
